In [1]:
#import necessary libraries
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences

def extract_features(file):
    #load the model
    model1 = VGG16()
    
    model = VGG16()
    #retstucture the model
    model1.layers.pop()
    
    model1= Model(inputs = model1.inputs, outputs = model1.layers[-1].output)
    #load the image
    image = load_img(file,target_size=(224,224))
    #convert image into array
    image = img_to_array(image)
    #reshape data for model
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    #prepare the image for VGG model
    image = preprocess_input(image)
    #get features of size (1,1049)
    feature = model1.predict(image)
    #get the prediction of the image(1,1000) accross al inputs
    yhat = model.predict(image)
    #convert proability to cals labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))
    return feature, label

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        #pad input
        sequence =  pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        #convert proability to integer
        yhat = argmax(yhat)
        #map integer to word
        print(yhat)
        word = word_id(yhat, tokenizer)
        #stop if we cannot map the word
        if word is None:
            break
        in_text += ' ' + word
        #stop in he end of the sentence
        if word == 'endseq':
            break
    return in_text

#map an integer to word
def word_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None




Using TensorFlow backend.
/home/sujans/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sujans/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sujans/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sujans/anaconda3/envs/dl/l

In [ ]:
from matplotlib import pyplot
import cv2 as cv 
from matplotlib.image import imread
import re
from imutils import build_montages
from imutils import paths
import numpy as np
import random
import matplotlib.pyplot as plt
from pickle import load
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model

# load the tokenizer
tokenizer = load(open('tokenizer.pkl','rb'))
# pre-define the max sequence length (from training)
max_length = 34
#load the model
model = load_model('model_11.h5')
# grab all image paths in the input directory and randomly sample them
imagePaths = list(paths.list_images('images/'))
random.shuffle(imagePaths)
imagePaths = imagePaths[:20]
#initialize the result
results = []
# loop over our sampled image paths
#etract features t=of the image
for p in imagePaths:
    #extract features t=of the image
    photo, label = extract_features(p)
    #generate the descriptions of the image
    description = generate_desc(model, tokenizer, photo, max_length)
    #replace start and end of the sentence
    description = description.replace('startseq','')
    description = description.replace('endseq','')
    #load original image
    orig = cv.imread(p)
    #set color
    color = (0, 0, 255)
    #resize the image
    orig = cv.resize(orig, (700, 700))
    #draw over image
    cv.putText(orig, label[1]+" "+str(np.round(label[2]*100,2))+'%', (3, 50), cv.FONT_HERSHEY_SIMPLEX, 0.8,color, 2)
    cv.putText(orig, description, (3, 680), cv.FONT_HERSHEY_SIMPLEX, 0.8,(0, 255, 0), 2)
    # add the output image to our list of results
    results.append(orig)
# create a montage using 450x450 "tiles" with 1 rows and 3 columns
montage = build_montages(results, (450, 450), (3, 1))[0]

# show the output montage
cv.imshow("Results", montage)
cv.waitKey(0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


soccer_ball (89.29%)
12
62
16
33
101
5
4
40
2
golden_retriever (89.81%)
8
6
31
32
4
40
2
Airedale (30.12%)
12
30
16
33
3
4
41
2
